## Export GitHub Commits to CSV

### Input

In [1]:
year = 2023
owner = 'johnnyoshika'
repo = 'testing'

### Setup

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### HTTP Request

In [3]:
import os
import requests

headers = {
    'Authorization': f'token {os.getenv("GITHUB_ACCESS_TOKEN")}',
    'Accept': 'application/vnd.github.v3+json',
}

def fetch_commits(url):
    response = requests.get(url, headers=headers)
    commits = response.json()
    next_page_url = None

    # Check if there is a 'Link' header and parse for next page URL
    if 'link' in response.headers:
        links = response.headers['link'].split(', ')
        next_link = [link for link in links if 'rel="next"' in link]
        if next_link:
            next_page_url = next_link[0].split(';')[0].strip('<>')

    return commits, next_page_url

### Export to CSV

In [4]:
from datetime import datetime
import csv

since = datetime(year, 1, 1).isoformat() + 'Z'
until = datetime(year + 1, 1, 1).isoformat() + 'Z'

base_url = f'https://api.github.com/repos/{owner}/{repo}/commits?since={since}&until={until}'

filename = f'{year}_{repo}_commits.csv'
filepath = os.path.join(os.path.expanduser('~'), 'Downloads', filename)

# Define the CSV header
fields = ['SHA', 'Author', 'Date', 'Message']

if os.path.exists(filepath):
    raise FileExistsError(f"The file '{filepath}' already exists. Please remove it first.")

with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)  # Write the CSV header
    
    next_url = base_url
    while next_url:
        commits, next_url = fetch_commits(next_url)
        for commit in commits:
            sha = commit['sha']
            author = commit['commit']['author']['name']
            date = commit['commit']['author']['date']
            message = commit['commit']['message']
            csvwriter.writerow([sha, author, date, message])

print(f"Commits saved to '{filepath}'.")

Commits saved to 'C:\Users\Johnny\Downloads\2023_testing_commits.csv'.
